In [2]:
import os, sys
os.chdir('/home/seigyo/Documents/pytorch/brain_decoder')
sys.path.append(os.pardir)
import numpy as np
from numpy.random import RandomState
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
import mne
from mne.io import concatenate_raws
from mymodule.utils import data_loader, evaluator
from mymodule.layers import LSTM, Residual_block, Res_net, Wavelet_cnn, NlayersSeqConvLSTM
from mymodule.trainer import Trainer
from sklearn.utils import shuffle
from tensorboardX import SummaryWriter
from load_data import get_data, get_data_multi, get_crops, get_crops_multi
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
substart=0
subend=1
for idx in range(substart, subend):
    X1, y1 = get_data(id=idx+1, event_code=[4,8,12], filter=[0.5, 30], t=[0., 4])
    X2, y2 = get_data(id=idx+1, event_code=[6,10,14], filter=[0.5, 30], t=[0., 4])    

Removing orphaned offset at the beginning of the file.
89 events found
Events id: [1 2 3]
45 matching events found
Not setting metadata
Loading data for 45 events and 641 original time points ...
0 bad epochs dropped
Removing orphaned offset at the beginning of the file.
89 events found
Events id: [1 2 3]
45 matching events found
Not setting metadata
Loading data for 45 events and 641 original time points ...
0 bad epochs dropped


In [36]:
id=1
event_code=[4,6,8,10,12,14]
filter=[0.5, 36]
t=[1, 4.1]
# 5,6,7,10,13,14 are codes for executed and imagined hands/feet
subject_id = id
event_codes = event_code

# This will download the files if you don't have them yet,
# and then return the paths to the files.
physionet_paths = mne.datasets.eegbci.load_data(subject_id, event_codes)

# Load each of the files
parts = [mne.io.read_raw_edf(path, preload=True,stim_channel='auto', verbose='WARNING')
         for path in physionet_paths]

# Concatenate them
raw = concatenate_raws(parts)

# bandpass filter
if filter != None:
    raw.filter(filter[0], filter[1], fir_design='firwin', skip_by_annotation='edge')
else:
    pass

# Find the events in this dataset
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

# Use only EEG channels
eeg_channel_inds = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Extract trials, only using EEG channels
epoched = mne.Epochs(raw, events, [2, 3], tmin=t[0], tmax=t[1], proj=False, picks=eeg_channel_inds,
                baseline=None, preload=True)
# change time length
# epochs_train = epochs.copy().crop(tmin=1., tmax=2.)


# Convert data from volt to millivolt
# Pytorch expects float32 for input and int64 for labels.
X = (epoched.get_data() * 1e6).astype(np.float32)
y = (epoched.events[:,2] - 2).astype(np.int64) #2,3 -> 0,1

Removing orphaned offset at the beginning of the file.
179 events found
Events id: [1 2 3]
90 matching events found
Not setting metadata
Loading data for 90 events and 497 original time points ...
0 bad epochs dropped


In [43]:
os.mkdir('accuracies')
np.save('accuracies/savetest.npy',y)

In [44]:
np.load('accuracies/task(rf)model(Conv)_acc.npy')

array([ 0.67391304,  0.51190476,  0.54347826,  0.60177866,  0.54347826])